In [2]:
!pip install numpy

Looking in indexes: https://artifacts.jpmchase.net/artifactory/api/pypi/pypi/simple
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     ----- ---------------------------------- 1.8/12.8 MB 7.2 MB/s eta 0:00:02
     ------------------------------- ------- 10.5/12.8 MB 23.0 MB/s eta 0:00:01
     ----------------------------------- --- 11.5/12.8 MB 22.0 MB/s eta 0:00:01
     ----------------------------------- --- 11.5/12.8 MB 22.0 MB/s eta 0:00:01
     ----------------------------------- --- 11.8/12.8 MB 12.6 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.4 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 9.0 MB/s eta 0:00:00


In [1]:
import numpy as np

In [2]:
def sigmoid(z):
  return 1.0 / (1 + np.exp(-z))

In [3]:
def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

In [4]:
def sigmoid_derivative(x):
    return x * (1 - x)

In [6]:
def initialized_network(input_size: int, hidden_layers: list, output_size: int):
    layers = [input_size] + hidden_layers + [output_size]
    weights = []
    biases = []
    print(f' initialize_network {layers=}')
    
    l = len(layers) - 1

    for i in range(l):
        
        weight = np.random.uniform(-1, 1, (layers[i], layers[i+1]))
        bias = np.random.uniform(-1, 1, (layers[i+1],))
        print(f' #{i+1}/{l} {weight=}')
        print(f'   {bias=}\n')
        weights.append(weight)
        biases.append(bias)

    return weights, biases
        

In [7]:
def train_network(td, w, b, lr=0.1, e=1000):
    for epoch in range(e):
        total_error = 0
        for input_val, expected_output in td:
            total_error += train_once(input_val, expected_output, lr, w, b)
        if epoch % 1000 ==0:
            print(f"Epoch {epoch}, Total Error: {total_error}")

    return w, b
            

In [8]:
def forward_pass(input_data, weights, biases):
    activations = [input_data]
    for i in range(len(weights)):
        # on error: likely due to the input shape not matching the input nodes.
        input_data = np.dot(input_data, weights[i]) + biases[i]
        input_data = sigmoid(input_data)
        activations.append(input_data)
    return activations

In [39]:

def forward_pass_one(input_data, weights, biases):
    # # Generalized forward pass through all layers
    # for i in range(len(weights)):
    #     input_data = np.dot(input_data, weights[i]) + biases[i]
    #     input_data = sigmoid(input_data)
    # return input_data
    return forward_pass(input_data, weights, biases)[-1]


In [9]:
def train_once(input_val, expected_output, learning_rate, weights, biases):
    """
    # Example of adjusting training data to include negative example flag
    training_data = [
        # Positive examples: ((input_features), (target_output, False))
        ((input_features_for_example_1), (target_output_for_example_1, False)),
        # Negative examples: ((input_features), (target_output, True))
        ((input_features_for_negative_example_1), (target_output_for_negative_example_1, True)),
    ]

    """
    # Forward pass: Store activations for each layer
    activations = forward_pass(input_val, weights, biases)

    # Backward pass
    # Calculate output error
    output_error = expected_output - activations[-1]
    total_error = np.sum(output_error ** 2)

    # Calculate gradient for output layer
    d_error = output_error * sigmoid_derivative(activations[-1])

    for i in reversed(range(len(weights))):
        # Calculate error for the current layer
        d_weights = np.outer(activations[i], d_error)
        d_biases = d_error

        # Update weights and biases
        weights[i] += learning_rate * d_weights
        # cannot be: `biases[i] += learning_rate * d_biases` due to numpy
        biases[i] = biases[i] + (learning_rate * d_biases)

        # Propagate the error backward
        if i > 0:
            d_error = np.dot(d_error, weights[i].T) * sigmoid_derivative(activations[i])
    return total_error

In [10]:
def dictify(word):
    thedict = {}
    for i, letter in enumerate(word):
        thedict[letter] = (i+1)/len(word)
    return thedict

In [207]:
bits = dictify("horsetrees")
bits

def word_to_ints(word):
    return [bits[x] for x in word]

In [208]:
tds = [
    (
        word_to_ints('horse'),
        word_to_ints('trees'),
    ),
    (
        
        word_to_ints('trees'),
        word_to_ints('horse'),
    )
]

In [210]:
ws, bs = initialized_network(5,[3],5)
train_network(tds, ws, bs)

 initialize_network layers=[5, 3, 5]
 #1/2 weight=array([[ 0.02238314,  0.23480097, -0.28986135],
       [ 0.25553781,  0.89402362, -0.04555032],
       [-0.03561692, -0.10750907, -0.87661902],
       [ 0.79351426, -0.73018797, -0.72513966],
       [-0.44394967, -0.53306918, -0.15993205]])
   bias=array([-0.59811013,  0.95912117,  0.17648279])

 #2/2 weight=array([[-0.37797228, -0.11013702,  0.73457582, -0.77582103, -0.67079482],
       [ 0.65086708, -0.76039068, -0.99236149, -0.11071205,  0.42717254],
       [ 0.96776451,  0.44553305,  0.11301805, -0.92965602, -0.02730808]])
   bias=array([ 0.21523055, -0.50006289, -0.24138141,  0.25086798,  0.35412338])

Epoch 0, Total Error: 1.6581484999726324


([array([[ 0.20861483,  0.8612228 , -0.89405924],
         [ 0.44949826,  1.50544208, -0.60898547],
         [ 0.08988556,  0.04403739, -0.84926408],
         [ 0.83510126, -1.00850676, -0.18852011],
         [-0.33869026, -0.53981451,  0.07794051]]),
  array([[-0.74670274,  0.08444119,  1.23538069,  0.1546361 ,  0.09081227],
         [-0.57957409, -1.42100667, -0.73732998,  0.8822574 ,  1.1066281 ],
         [ 1.36974164,  1.01912294,  0.41500308, -0.67538103,  0.24994701]])],
 [array([-0.52082255,  0.80908742,  0.58411022]),
  array([-0.26239758,  0.24920876,  0.94582456,  2.17071365,  1.95493525])])

In [213]:
# cat_numbers = (bits["c"], bits["a"], bits["t"])
# dog_numbers = (bits["d"], bits["o"], bits["g"])
# fp = forward_pass(cat_numbers, ws, bs)
stib = {v:k for k,v in bits.items()}
print(f"stib: {stib}")
print(f"fp: {fp}")

stib: {0.1: 'h', 0.2: 'o', 0.7: 'r', 1.0: 's', 0.9: 'e', 0.6: 't'}
fp: [(0.16666666666666666, 0.3333333333333333, 0.5), array([0.67082232, 0.42880837, 0.74335173]), array([0.6636555 , 0.83468608, 0.93690201])]


In [176]:
# Needs work
def closest_match(value, values_in, values_out):
    fp = forward_pass(value, ws, bs)
    
    for i in range(len(ws)):
        value = np.dot(value, ws[i]) + bs[i]
        value = sigmoid(value)

        
    final_output = fp[0]
    closest_numb = min(values_in.values(), key=lambda x: abs(x - final_output))
    predicted_letter = values_out[closest_numb]
    return predicted_letter

In [214]:
def closest_match(value, values_in, values_out):
    input_val = np.array(value)
    fp = forward_pass(input_val, ws, bs)[-1] # last Activation layer (-1)
    return flat_word(fp, ws, bs, values_in, values_out)

In [217]:
a = closest_match(word_to_ints('horse'), bits, stib)

In [195]:
def predict(input_val, weights, biases, letter_to_num, num_to_letter):
    # Ensure input is in the correct format (numpy array)
    input_val = np.array(input_val)
    v = forward_pass_one(input_val, weights, biases)
    # Generalized forward pass through all layers
    # for i in range(len(weights)):
    #     input_val = np.dot(input_val, weights[i]) + biases[i]
    #     input_val = sigmoid(input_val)
    # assert v == input_val
    # Convert the final output to the closest letter
    # Here, we're assuming the output is a single number. Adjust as needed for multiple outputs.
    # final_output = v[0]  # Assuming single output for simplicity
    return flat_word(v, weights, biases, letter_to_num, num_to_letter)

In [196]:
def flat_word(v, weights, biases, letter_to_num, num_to_letter):
    res = ''
    for final_output in v:
        closest_num = min(letter_to_num.values(), key=lambda x: abs(x - final_output))
        predicted_letter = num_to_letter[closest_num]
        # print(predicted_letter)
        res += predicted_letter
    return res

In [231]:
a = predict(word_to_ints('trees'), ws, bs, bits, stib)
b = predict(word_to_ints('horse'), ws, bs, bits, stib)

print(a,b)

horse trees


In [227]:
ws, bs

([array([[ 0.35952002,  1.89319678, -2.05981393],
         [ 0.60275219,  2.49845558, -1.71827859],
         [ 0.16574935,  0.19968782, -0.9429196 ],
         [ 0.82887742, -1.61229844,  0.62053882],
         [-0.28784026, -0.67627193,  0.34165137]]),
  array([[-1.03169847, -0.11772165,  1.13864036,  0.30949097,  0.14740841],
         [-2.07193428, -2.5258841 , -1.42559793,  1.17918828,  0.85366211],
         [ 2.58302613,  1.91545206,  1.05017897, -0.74867268,  0.65953215]])],
 [array([-0.4973351 ,  0.41948263,  1.1487259 ]),
  array([-0.26667272,  0.26527875,  1.00729854,  2.37421736,  2.16536446])])

In [199]:
fp

[(0.16666666666666666, 0.3333333333333333, 0.5),
 array([0.67082232, 0.42880837, 0.74335173]),
 array([0.6636555 , 0.83468608, 0.93690201])]

In [230]:
train_network(tds, ws, bs)

Epoch 0, Total Error: 0.006463620202123889


([array([[ 0.35665926,  1.92110013, -2.14621473],
         [ 0.60110209,  2.52512802, -1.80198042],
         [ 0.1725954 ,  0.20272513, -0.95966672],
         [ 0.84179829, -1.63043443,  0.66534848],
         [-0.27775862, -0.6815233 ,  0.34812214]]),
  array([[-1.05556748, -0.11562765,  1.13725498,  0.33218353,  0.09093537],
         [-2.16332258, -2.49218267, -1.51560058,  1.28605247,  0.5994567 ],
         [ 2.65436387,  1.87716938,  1.15970735, -0.84151115,  0.87607124]])],
 [array([-0.48522851,  0.40717349,  1.1757156 ]),
  array([-0.28085168,  0.25817313,  1.03412848,  2.38125345,  2.14407674])])